In [1]:
# Adding new data to the existing database
# Filling the Database 
import re
from dateutil import parser
from datetime import datetime
from fuzzywuzzy import fuzz, process
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from docx import Document
import os
import pandas as pd

week = '08_14Jan2025'
database = pd.read_csv(r'data\Databasev04_18Dec2024_07Jan2025.csv')
new = pd.DataFrame(columns=database.columns)
new.columns
folder = r'E:\Sanaa Center\02_ Weekly Reports\15_ (08- 14) Jan - 2025\Reports\Translated Files'

event_num = ['Event', 'event']
frontlines = ['Front Name', 'Front name', 'front name', 'front Name', 'Frontline Name', 'Frontline name', 'frontline name', 'frontline Name']
indx = 0

files = [file for file in os.listdir(folder) if file[-5: ]=='.docx']
print('Reading the data ................')
for file in files:
    doc = Document(os.path.join(folder, file))
    table_num = 0
    try:
        for table_idx, table in enumerate(doc.tables):
            table_num = table_num +1
            if any(name in table.rows[0].cells[1].text.strip() for name in event_num):
                # This table is right to left table
                if table.rows[2].cells[0].text.strip() == '':
                    pass
                else:
                    new.loc[indx, 'Date'] = table.rows[1].cells[0].text.strip() # Date
                    new.loc[indx, 'Event_num'] = table.rows[0].cells[0].text.strip() # Event number
                    new.loc[indx, 'Type'] = None  # Type
                    new.loc[indx, 'SubType'] = None  # SubType
                    new.loc[indx, 'Governorate'] = file # Governorate
                    new.loc[indx, 'District'] = None # District
                    
                    try:
                        rows_titles = [value.cells[1].text.strip() for value in table.rows]
                    except:
                        pass

                    # Filling the frontline column
                    if any(name in rows_titles for name in frontlines):
                        new.loc[indx, 'Frontline'] = table.rows[2].cells[0].text.strip() # Frontline
                    else:
                        new.loc[indx, 'Frontline'] = None # Frontline

                    # Filling the coordinates column
                    cord_indx = [index for index, title in enumerate(rows_titles) if 'coordinates' in title or 'Coordinates' in title]
                    if cord_indx != []:
                        new.loc[indx, 'Coordinates'] = table.rows[cord_indx[0]].cells[0].text.strip()   

                    # Filling the location column
                    loc_indx = [index for index, title in enumerate(rows_titles) if 'location' in title or 'Location' in title]
                    if loc_indx != []:
                        new.loc[indx, 'Location'] = table.rows[loc_indx[0]].cells[0].text.strip() 

                    # Filling the side 1 column
                    s01_indx = [index for index, title in enumerate(rows_titles) if 'Side 1' in title or 'side 1' in title]
                    if s01_indx != []:
                        new.loc[indx, 'Side_1'] = table.rows[s01_indx[0]].cells[0].text.strip()  
                    
                    # Filling the side 2 column
                    s02_indx = [index for index, title in enumerate(rows_titles) if 'Side 2' in title or 'Side 2' in title]
                    if s02_indx != []:
                        new.loc[indx, 'Side_2'] = table.rows[s02_indx[0]].cells[0].text.strip()  
                    
                    # Filling the Details column
                    det_indx = [index for index, title in enumerate(rows_titles) if ('detail' in title or 'Details' in title or 'details' in title
                                or 'Detailed' in title or 'detailed' in title) and 'location' not in title ]
                    if det_indx != []:
                        new.loc[indx, 'Details'] = table.rows[det_indx[-1]].cells[0].text.strip()  

                    # Filling the Casualties_1 column
                    cas01_indx = [index for index, title in enumerate(rows_titles) if '1 Casualties' in title or '1 casualties' in title or 'Casualties 1' in title
                                or 'Casualties 1' in title]
                    if cas01_indx != []:
                        new.loc[indx, 'Side_1_killes'] = table.rows[cas01_indx[-1]].cells[0].text.strip() 

                    # Filling the Casualties_2 column
                    cas02_indx = [index for index, title in enumerate(rows_titles) if '2 Casualties' in title or '2 casualties' in title or 'Casualties 2' in title
                                or 'Casualties 2' in title]
                    if cas02_indx != []:
                        new.loc[indx, 'Side_2_killes'] = table.rows[cas02_indx[-1]].cells[0].text.strip()   

                    # Filling the Coordinator column
                    cordator_indx = [index for index, title in enumerate(rows_titles) if 'Coordinator' in title or 'coordinator' in title ]
                    if cordator_indx != []:
                        new.loc[indx, 'Cord_analysis'] = table.rows[cordator_indx[-1]].cells[0].text.strip()

                    # Filling the Source column
                    src_indx = [index for index, title in enumerate(rows_titles) if 'Source' in title or 'source' in title ]
                    if src_indx != []:
                        new.loc[indx, 'Source'] = table.rows[src_indx[-1]].cells[0].text.strip()     

                    new.loc[indx, 'Side_1_wounds'] = None        
                    new.loc[indx, 'Side_2_wounds'] = None   
                    new.loc[indx, 'E'] = None   
                    new.loc[indx, 'N'] = None    
                    new.loc[indx, 'ADM1_EN'] = None       

                    indx = indx + 1

            elif any(name in table.rows[0].cells[0].text.strip() for name in event_num):
                # This table is left to right table
                if table.rows[2].cells[1].text.strip() == '':
                    pass
                else:
                    new.loc[indx, 'Date'] = table.rows[1].cells[1].text.strip() # Date
                    new.loc[indx, 'Event_num'] = table.rows[0].cells[1].text.strip() # Event number
                    new.loc[indx, 'Type'] = None  # Type
                    new.loc[indx, 'SubType'] = None  # SubType
                    new.loc[indx, 'Governorate'] = file # Governorate
                    new.loc[indx, 'District'] = None # District
                    
                    try:
                        rows_titles = [value.cells[0].text.strip() for value in table.rows]
                    except:
                        pass

                    # Filling the frontline column
                    if any(name in rows_titles for name in frontlines):
                        new.loc[indx, 'Frontline'] = table.rows[2].cells[1].text.strip() # Frontline
                    else:
                        new.loc[indx, 'Frontline'] = None # Frontline

                    # Filling the coordinates column
                    cord_indx = [index for index, title in enumerate(rows_titles) if 'coordinates' in title or 'Coordinates' in title]
                    if cord_indx != []:
                        new.loc[indx, 'Coordinates'] = table.rows[cord_indx[0]].cells[1].text.strip()   

                    # Filling the location column
                    loc_indx = [index for index, title in enumerate(rows_titles) if 'location' in title or 'Location' in title]
                    if loc_indx != []:
                        new.loc[indx, 'Location'] = table.rows[loc_indx[0]].cells[1].text.strip() 

                    # Filling the side 1 column
                    s01_indx = [index for index, title in enumerate(rows_titles) if 'Side 1' in title or 'side 1' in title]
                    if s01_indx != []:
                        new.loc[indx, 'Side_1'] = table.rows[s01_indx[0]].cells[1].text.strip()  
                    
                    # Filling the side 2 column
                    s02_indx = [index for index, title in enumerate(rows_titles) if 'Side 2' in title or 'Side 2' in title]
                    if s02_indx != []:
                        new.loc[indx, 'Side_2'] = table.rows[s02_indx[0]].cells[1].text.strip()  
                    
                    # Filling the Details column
                    det_indx = [index for index, title in enumerate(rows_titles) if ('detail' in title or 'Details' in title or 'details' in title
                                or 'Detailed' in title or 'detailed' in title) and 'location' not in title]
                    if det_indx != []:
                        new.loc[indx, 'Details'] = table.rows[det_indx[-1]].cells[1].text.strip()  

                    # Filling the Casualties_1 column
                    cas01_indx = [index for index, title in enumerate(rows_titles) if '1 Casualties' in title or '1 casualties' in title or 'Casualties 1' in title
                                or 'Casualties 1' in title]
                    if cas01_indx != []:
                        new.loc[indx, 'Side_1_killes'] = table.rows[cas01_indx[-1]].cells[1].text.strip() 

                    # Filling the Casualties_2 column
                    cas02_indx = [index for index, title in enumerate(rows_titles) if '2 Casualties' in title or '2 casualties' in title or 'Casualties 2' in title
                                or 'Casualties 2' in title]
                    if cas02_indx != []:
                        new.loc[indx, 'Side_2_killes'] = table.rows[cas02_indx[-1]].cells[1].text.strip()   

                    # Filling the Coordinator column
                    cordator_indx = [index for index, title in enumerate(rows_titles) if 'Coordinator' in title or 'coordinator' in title ]
                    if cordator_indx != []:
                        new.loc[indx, 'Cord_analysis'] = table.rows[cordator_indx[-1]].cells[1].text.strip()

                    # Filling the Source column
                    src_indx = [index for index, title in enumerate(rows_titles) if 'Source' in title or 'source' in title ]
                    if src_indx != []:
                        new.loc[indx, 'Source'] = table.rows[src_indx[-1]].cells[1].text.strip()     

                    new.loc[indx, 'Side_1_wounds'] = None        
                    new.loc[indx, 'Side_2_wounds'] = None   
                    new.loc[indx, 'E'] = None   
                    new.loc[indx, 'N'] = None    
                    new.loc[indx, 'ADM1_EN'] = None       

                    indx = indx + 1
            
            elif 'Houthi' in table.rows[0].cells[0].text.strip() and len(table.rows)==9:
                if any(char.isalnum() for char in table.rows[1].cells[0].text.strip()):
                    new.loc[indx, 'Governorate'] = file # Governorate
                    new.loc[indx, 'SubType'] = 'US - Israeli Airstrike'  # Type
                    new.loc[indx, 'Date'] = f'{table.rows[1].cells[0].text.strip()} (Houthi Reporting) \n {table.rows[1].cells[1].text.strip()} (CENTCOM Reporting)' # Date
                    new.loc[indx, 'Location'] = table.rows[2].cells[0].text.strip() 
                    new.loc[indx, 'Details'] = f'(Houthi Reporting): {table.rows[4].cells[0].text.strip()} \n \n (CENTCOM Reporting): {table.rows[4].cells[1].text.strip()}'
                    new.loc[indx, 'Cord_analysis'] = table.rows[5].cells[0].text.strip()
                    new.loc[indx, 'Side_1_killes'] = f'(Houthi Reporting): {table.rows[6].cells[0].text.strip()} \n \n (CENTCOM Reporting): {table.rows[6].cells[1].text.strip()}'
                    
                    for cell in table.rows[7].cells:
                        if "." in cell.text.strip() or '°' in cell.text.strip():
                            new.loc[indx, 'Coordinates'] = cell.text.strip()
                            break 
                    
                    new.loc[indx, 'Source'] = f'(Houthi Reporting): {table.rows[8].cells[0].text.strip()} \n \n (CENTCOM Reporting): {table.rows[8].cells[1].text.strip()}'
                    indx = indx + 1
                
            else:
                pass
    except:
        print('--------------------')
        print("Error")
        print(file)
        print(table_num)
        print('--------------------')

print('Done')
print('       ')


# Adjusting Governorates names
Governorates = ["Al-Dhalea", "Abyan", "Al-Bayda", "Taiz", "Al-Jawf", "Hudaydah", "Hadramawt", "Shabwa", "Sa'ada", "Sana'a", "Aden", "Lahj", "Marib", "Al-Mahra"]
print('Editing the governorates names ................')
counter = 0
for governorate in Governorates:
    target_word = governorate
    for row in range(len(new)):
        name = new.loc[row, 'Governorate']
        words = re.split(r'\W+', name) 
        similar_words = [word for word in words if fuzz.ratio(target_word, word) > 70]  
        if similar_words != []:
            new.loc[row, 'Governorate'] = governorate

print('Done')
print('       ')



# Adjusting Dates format
print('Editing the dates format ................')
date_patterns = [
    r"\b\d{4}-\d{2}-\d{2}\b",              # YYYY-MM-DD
    r"\b\d{2}/\d{2}/\d{4}\b",              # DD/MM/YYYY or MM/DD/YYYY (ambiguous)
    r"\b\d{2}-\d{2}-\d{4}\b",              # MM-DD-YYYY
    r"\b\d{4}/\d{2}/\d{2}\b",              # YYYY/MM/DD
    r"\b\w+\s\d{1,2},\s\d{4}\b",           # Month DD, YYYY (e.g., March 15, 2022)
    r"\b\w{3}\s\d{1,2},\s\d{4}\b",         # Abbreviated Month DD, YYYY (e.g., Feb 20, 2024)
    r"\b\d{1,2}\s*/\s*\w+\s*/\s*\d{4}\b",  # DD / Month / YYYY (e.g., 27 / February / 2023)
    r"\b\d{4}\s\w{3}\s\d{1,2}\b",          # YYYY Month DD (e.g., 2023 Feb 28)
    r"\b\d{1,2}\s\w+\s\d{4}\b",            # DD Month YYYY (e.g., 9 January 2023)
    r"\b\w+\s\d{1,2}\s*-\s*\d{1,2},\s\d{4}\b",  # Month DD - DD, YYYY (e.g., May 28 - 31, 2024)
    r"\b\w+\s\d{1,2}\b",                   # Month DD (e.g., December 13)
    r"\b\d{1,2}\s\w+,\s\d{4}\b",           # DD Month, YYYY (e.g., 30 July, 2024)
    r"\b\d{1,2}\s*-\s*\w+\s\d{4}\b"        # DD-Month-YYYY range
]

def find_and_convert_dates(text, default_year=2024, ambiguous_format="MDY"):
    dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, text)
        for match in matches:
            try:
                # Special handling for date ranges like "May 28 - 31, 2024"
                if re.match(r"\b\w+\s\d{1,2}\s*-\s*\d{1,2},\s\d{4}\b", match):
                    # Extract only the first day (e.g., "May 28, 2024" from "May 28 - 31, 2024")
                    first_day = re.sub(r"-\s*\d{1,2}", "", match)
                    parsed_date = parser.parse(first_day)
                elif re.match(r"\b\w+\s\d{1,2}\b", match):
                    # Add the default year for dates without a year (e.g., "December 13" -> "December 13, 2024")
                    date_with_year = f"{match}, {default_year}"
                    parsed_date = parser.parse(date_with_year)
                elif re.match(r"\b\d{2}/\d{2}/\d{4}\b", match):
                    # Handle ambiguous MM/DD/YYYY or DD/MM/YYYY formats
                    day, month, year = match.split('/')
                    if ambiguous_format == "DMY":
                        parsed_date = datetime.strptime(match, "%d/%m/%Y")
                    else:  # Default to "MDY"
                        parsed_date = datetime.strptime(match, "%m/%d/%Y")
                else:
                    # For other formats, use the dateutil parser
                    parsed_date = parser.parse(match)
                
                dates.append(parsed_date)
            except ValueError:
                pass  # Ignore unparseable dates

    # Convert dates to MM/DD/YY format
    formatted_dates = [date.strftime("%m/%d/%y") for date in dates]
    return formatted_dates



counter = 0
for row in range(len(new)):
    date = new.loc[row, 'Date']
    if date:
        found_dates = find_and_convert_dates(date)
        if found_dates != []:
            new.loc[row, 'Date'] = found_dates[0]
        else:
            if ('/') in date:
                if int(date[0:2])>12:
                    corr_date = f'{date[3:5]}/{date[0:2]}/{date[6: ]}'
                    new.loc[row, 'Date'] = corr_date
                    

                
    else:
        pass
print('Done')
print('       ')


# Using the trained model for categorizing events
print('Catogrizing the events ................')
tokenizer = AutoTokenizer.from_pretrained(r"E:\Sanaa Center\04_ Other Tasks\02_ All weekly reports database\Codes\Saved Models\v03")
model = AutoModelForSequenceClassification.from_pretrained(r"E:\Sanaa Center\04_ Other Tasks\02_ All weekly reports database\Codes\Saved Models\v03")

counter = 0
for row in range(len(new)):
    text = new.loc[row, 'Details']
    if not text:
        new.loc[indx, 'Details'] = None
    else:
        print('------- Categorzizing -------')
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        logits = outputs.logits

        predicted_class = logits.argmax(dim=1).item()
        
        if predicted_class == 0:
            predicted_class = 'Landmine'
        elif predicted_class == 1:
            predicted_class= 'Civilian assualt'
        elif predicted_class == 2:
            predicted_class= 'US - Israeli Airstrike'
        elif predicted_class == 3:
            predicted_class= 'Meeting event'
        elif predicted_class == 4:
            predicted_class= 'Clash event with Al-Qaeda'
        elif predicted_class == 5:
            predicted_class= 'Clash event with Houthi'
        elif predicted_class == 6:
            predicted_class= 'Normal clash event'
        elif predicted_class == 7:
            predicted_class= 'Drone & Missile'
        elif predicted_class == 8:
            predicted_class= 'Normal event'
        elif predicted_class == 9:
            predicted_class= 'Assassination & Arrest'
        elif predicted_class == 10:
            predicted_class= 'Vessels attack'
        new.loc[row, 'SubType'] = predicted_class
        counter = counter +1 
print('Done')
print('       ')


#Adding the district names
print('Adding the districts names ................')
districts = pd.read_excel(r"E:\Sanaa Center\04_ Other Tasks\02_ All weekly reports database\Codes\Districts_with_coords.xlsx")

def assign_districts(texts, districts):
    matches = []
    for text in texts:
        # Find the best match from districts for each text
        match, score = process.extractOne(text, districts, scorer=fuzz.token_set_ratio)
        if score > 65:  # Define a threshold for a good match
            matches.append((match, score))
        else:
            matches.append((None, 0))
    return matches


for index, row in new.iterrows():
    if pd.isna(row[6]):
        new.loc[index, 'District'] = None
    else:
        if 'Al ' in row[6]:
            row[6].replace('Al ', 'Al-')
        elif 'Dar ' in row[6]:
            row[6].replace('Dar ', 'Dar-')

        try: 
            texts = row[6].split(' ') 
            targ_districts = districts[districts['Governorate']==row[4]]
            targ_districts = list(targ_districts['District'])
            assigned_districts = assign_districts(texts, targ_districts)

            # Display results
            score_list = []
            for text, (district, score) in zip(texts, assigned_districts):
                score_list.append(score)
            for text, (district, score) in zip(texts, assigned_districts):
                if score == max(score_list):
                    new.loc[index, 'District'] = district
        except:
            new.loc[index, 'District'] = None

print('Done')
print('       ')


# Editing the coordinates
print('Editing the coordinates ................')
def extract_coordinates_data (text):
    float_pattern = r"(?<![°'])[-+]?\d*\.\d+|(?<![°'])[-+]?\d+(?![°'\"])"
    dms_pattern = r"\d{1,3}°\d{1,2}'\d{1,2}(?:\.\d+)?\"\s?[NSWE]"

    # Function to extract float numbers
    def extract_floats(text):
        return list(re.findall(float_pattern, text))
    # Function to extract DMS coordinates
    def extract_dms(text):
        return re.findall(dms_pattern, text)

    floats = extract_floats(text)
    for value in floats:
        if len(value) < 5:
            floats = [x for x in floats if x != value]
        else:
            pass


    dms = extract_dms(text)
    corrected_dms = []
    for dms_value in dms:
        corrected_dms.append(float(dms_value[0:2]) + float(int(dms_value[3:5])/60) + float(int(dms_value[6:8])/3600))

    coord = floats + corrected_dms
    

    return coord

for index, row in new.iterrows():
    if row[7] == '':
        filtered = districts[(districts['Governorate'] == row[4]) & (districts['District'] == row[5])]
        try:
            E = filtered['E'].values[0]
            N = filtered['N'].values[0]
            
            new.loc[index, 'E'] = float(E)
            new.loc[index, 'N'] = float(N)

            new.loc[index, 'Coordinates'] = f'{float(E)},{float(N)}'

        except:
            new.loc[index, 'N'] = None
            new.loc[index, 'E'] = None

    else:
        if type(row[7]) == float:
            new.loc[index, 'Coordinates'] = None
        else:
            coords = extract_coordinates_data(row[7])
            if len(coords) > 1:
                coords = coords[0: 2]
                if float(coords[0]) > 30:
                    new.loc[index, 'E'] = float(coords[0]) 
                    new.loc[index, 'N'] = float(coords[1])

                    new.loc[index, 'Coordinates'] = f'{float(coords[0])},{float(coords[1])}'
                else:
                    new.loc[index, 'E'] = float(coords[1])
                    new.loc[index, 'N'] = float(coords[0])

                    new.loc[index, 'Coordinates'] = f'{float(coords[1])},{float(coords[0])}'
            else:
                new.loc[index, 'Coordinates'] = None
    print('Done')
print('       ')

#new.to_excel(r'check.xlsx', index=False)
#new = pd.read_excel(r'check.xlsx')

#Editing the dates of the database of the web
print('Editing the dates 2nd time ................')
import datetime
formated_dates = []
dates = list(new['Date'].values)
index = 0
for date in dates:
    if pd.isna(date):
        new.loc[index, 'Date'] = None
        index = index + 1
    else:
        try:
            date_split = date.split(' ')
            if '00:00' in date_split:
                date_split.remove('00:00')
                date = date_split[0]
            date = datetime.datetime.strptime(date, '%m/%d/%y').strftime("%d/%m/%Y")
        except:
            date = datetime.datetime.strptime(date, '%m/%d/%Y').strftime("%d/%m/%Y")
        new.loc[index, 'Date'] = date
        index = index + 1

print('Done')
print('       ')



#Adding the new data to the database
print('Adding the new data to the database ................')
database = pd.concat([database, new])
database.to_csv(rf'data\Databasev04_{week}.csv', index=False, encoding='utf-8')
print('Done')
print('       ')

#Adding the new database to the web page 
print('Adding the new database to the web page ................')
import json
database = pd.read_csv(rf'data\Databasev04_{week}.csv')
data_list = database.to_dict(orient="records")
js_variable_name = "eventsData"
js_file = r"Data\Database_v02.js"  
with open(js_file, "w") as f:
    f.write(f"const {js_variable_name} = ")
    json.dump(data_list, f, indent=4)  
    f.write(";")  
print('Done')
print('       ')


print('       ')
print('All done')

Reading the data ................
Done
       
Editing the governorates names ................
Done
       
Editing the dates format ................
Done
       
Catogrizing the events ................
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzizing -------
------- Categorzi

Done
       
Editing the coordinates ................
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
       
Editing the dates 2nd time ................
Done
       
Adding the new data to the database ................
Done
       
Adding the new database to the web page ................
Done
       
       
All done
